#CDIA:::AAI - Greedy Best-First Search Methods-- SHORTEST PATH PROBLEM

Ibai Laña

In this notebook we will write a generalized version of Greedy Best First Search methods that can work with any problem once it is formulated. Then, the key of the solution will be generating proper problem formulations and the proper heuristic function.


## Frontier operation Functions
In GBFS methods the extraction is not based on their arrival to the queue order, but on the similarity to the heuristic function.



## PROBLEM FORMULATION
We will have a couple of evaluation functions based on different heuristics for the P8 problem and check how them affect to the solution


In [35]:
import numpy as np
class ProblemP8 ():
    #attributes of the class are empty
    name = ""
    initial_state = {}
    goal_state = {}
    actions = []
    rows = 3
    columns= 3

    def __init__(self): ## init method is the constructor.

        # normally we would go with a random initialization
        numbers =np.array([1,2,5,3,4,8,0,6,7])##--->> this configuration only requires depth 6 solution

        # numbers =np.array([1,2,5,3,6,7,0,4,8]) #---<>> depth 11 solution

        # numbers =np.array([1,2,5,4,6,7,3,8,0]) ## ->>>> higher depth

        self.name ="8PuzzleProblem"
        self.initial_state = numbers.reshape(self.rows, self.columns)
        self.goal_state = np.arange(self.rows*self.columns).reshape(self.rows, self.columns)
        self.actions = ["Up", "Down", "Left", "Right"]


    #we need this in case we want other size
    def set_row_cols (self, rows, cols):
        self.rows=rows
        self.columns=cols

    def is_final_state(self, state):
        return np.array_equal(state, self.goal_state)

    #####
    def is_applicable (self, state, action):
        #find where is 0 in the state to know if the change is applicable:
        row = np.where(state==0)[0][0] +1
        col = np.where(state==0)[1][0] +1
        if action == "Up":
          return row!=1
        if action == "Down":
          return row!=self.rows
        if action == "Left":
          return col!=1
        if action == "Right":
          return col!=self.columns


    # this function should exist for all problems, but its internal operation should be changed per problem
    # in this case, the applicability is checked in the expansion function, so the effect of a change is always produced
    def effect (self, state, action):
        row = np.where(state==0)[0][0]
        col = np.where(state==0)[1][0]
        result = state.copy()
        if action == "Up":
            result[row-1,col]=state[row,col]
            result[row,col]=state[row-1,col]

        if action == "Down":
          result[row+1,col]=state[row,col]
          result[row,col]=state[row+1,col]

        if action == "Left":
          result[row,col-1]=state[row,col]
          result[row,col]=state[row,col-1]

        if action == "Right":
          result[row,col+1]=state[row,col]
          result[row,col]=state[row,col+1]

        return result


  ## COST WILL BE 1 FOR EACH MOVED TILE
    def get_cost(self, action, state):
        return 1
  ### but the evaluation should be some function that tells us how different is the solution

    def manhattan_distance(self,state, goal_state): #h2
      manhattan = 0
      for i, j in zip (range(3), range(3)):
        state_value = state[i][j]
        position = np.where(np.array(goal_state)==state_value)
        manhattan_i_j = np.abs(position[0][0] - i) + np.abs(position[1][0] - j)
        manhattan+=manhattan_i_j
      return manhattan

    def wrong_positioned_tiles(self, state, goal_state): #h1
      return sum(sum(state!=self.goal_state))

    def get_evaluation (self, state):
      # return self.wrong_positioned_tiles (state, self.goal_state) #h1
      return self.manhattan_distance(state, self.goal_state)#h2


In [37]:

def print_results (res):
  print ("METHOD: {} \n Final_state: {} \n Final Status: {} \n Maximum frontier size: {}  \n Maximum Depth: {} \n Iterations reached: {}  \n COST:{}".format(res["method"], res["final_state"]["actions"], res["status"], res["max_frontier"], res["max_depth"], res["iterations"],res["final_state"]["cost"]))

problem = ProblemP8()
print(problem.initial_state)

print (problem.get_evaluation(problem.initial_state))
# print (problem.get_evaluation2(problem.initial_state))

[[1 2 5]
 [3 4 8]
 [0 6 7]]
2


# INFORMED SEARCH




## A*


In [31]:
def expand_a_star (node, problem):
    new_nodes = []
    possible_actions = problem.actions
    for action in possible_actions:
        if problem.is_applicable(node["state"], action):
            new_state= problem.effect (node["state"], action)
            new_node = {}
            new_node["state"]=new_state
            new_node["parent_node"]=node
            new_node["actions"]=node["actions"] + [action]
            new_node["cost"]=node["cost"] + problem.get_cost(action, node["state"])
            new_node["depth"]=node["depth"]+1
            ##### REWRITE THE EVALUATION
            new_node["evaluation"]=  new_node["cost"] + problem.get_evaluation(new_state)
            new_nodes.append (new_node)
    return new_nodes

In [32]:
def A_star(problem, iteration_limit):
    # result dictionary
    result = {"method":"A*", "final_state":[], "status":"No nodes in the frontier. No solution possible.",
             "max_frontier":0, "max_depth":0, "iterations":0}

    # 1. problem definition
    # problem = Problem()
    initial_node={"state":problem.initial_state, "parent_node":{}, "actions":[], "cost":0, "depth":0, "evaluation":problem.get_evaluation(problem.initial_state)}
    frontier = []

     # 2. add node to frontier
    frontier.append(initial_node)

    # 3. start exploring and expanding the frontier
    iterations=1
    while len (frontier)>0: #if we have elements in the frontier...

        #control if we have reached the iteration limit, stop
        if iterations>iteration_limit:
            result["status"] = "Maximum number of iterations reached"
            break

         # 3.1. get first element of frontier and delete it
        node = frontier[0]


        frontier = frontier[1:]
        # 3.2 check if it is final state:
        if problem.is_final_state (node["state"]):
            result["status"]="Solution Found."
            break #we end while. state will remain this last state computed, and sequence of actions will have all states.

        # 3.3 if it is not final, expand and add to the frontier

        new_nodes = expand_a_star(node, problem)
        for n in new_nodes:
            frontier.append(n)

        # sorting by ascending order this will lead to
        frontier = sorted(frontier, key=lambda field: field['evaluation'])

        # we compute the maximum size of frontier: the previous one or the current if it is bigger
        result["max_frontier"]=max(result["max_frontier"],len (frontier))
         # we compute the maximum depth: the previous one or the current if it is bigger
        result["max_depth"]=max(result["max_depth"], node["depth"])
         # we update the iterations count
        result["iterations"]= iterations

        iterations+=1
      #loop keeps running until no more nodes available or final state obtained

    result["final_state"] = node
    return(result)

# COMPARISON


##P8 PROBLEM WITH A*
HEURISTIC: MANHATTAN

HEURISTIC: BAD POSITIONED TILES





In [38]:
a_star_= A_star(problem, iteration_limit=12000)
print_results (a_star_ )

METHOD: A* 
 Final_state: ['Right', 'Right', 'Up', 'Up', 'Left', 'Left'] 
 Final Status: Solution Found. 
 Maximum frontier size: 57  
 Maximum Depth: 5 
 Iterations reached: 37  
 COST:6
